# Textklassifikation

In dieser Übung werden wir das *[20newsgroups](https://scikit-learn.org/stable/datasets/real_world.html#newsgroups-dataset)* Datenset benutzen um ein besseres Verständnis für Kernelmethoden und Klassifikation zu bekommen. Unser Ziel ist es, eine Reihe von Nachrichtenmeldungen 11 unterschiedlichen Kategorien zuzuordnen. Wir gehen dabei in zwei Schritten vor:
- Vorbereitung der Daten mittels Feature Extraction/Vectorization
- Klassifikation mittels Linear/Kernel SVM (Kapitel 4.2.3)

In [ ]:
# Stelle sicher, dass das Internet (unter Settings auf der rechten Seite) für das 
# Notebook aktiviert ist, damit das Herunterladen der Daten funktioniert.

from sklearn.datasets import fetch_20newsgroups

categories = [
    'alt.atheism',
    'comp.graphics',
    'misc.forsale',
    'rec.autos',
    'rec.motorcycles',
    'sci.electronics',
    'sci.space',
    'soc.religion.christian',
    'talk.politics.guns',
    'talk.politics.mideast',
    'talk.politics.misc',
]

data_train = fetch_20newsgroups(subset="train", shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'), categories=categories)
data_test = fetch_20newsgroups(subset="test", shuffle=True, random_state=42, remove=('headers', 'footers', 'quotes'), categories=categories)

In [ ]:
import numpy as np
X_train, y_train = data_train.data, data_train.target
X_test, y_test = data_test.data, data_test.target
len(X_train), len(y_train), len(np.unique(y_test))

Das Trainingsset enthält 11315 Nachrichtenmeldungen, welche 20 verschiedenen Kategorien zugewiesen sind. Diese Kategorien finden im Attribut `target_names`. Wir verwenden allerdings nur 11 dieser Kategorien, damit das Datenset nicht zu groß und die Rechenzeiten nicht zu lange werden. Das gibt uns 6200 Texte.

In [ ]:
data_train.target_names

Das Targetset `y_train` ist einfach ein *numpy array*, `X_test` ist jedoch eine Python Liste:

In [ ]:
type(X_train), type(y_train)

Wenn wir das erste Element ansehen, dann stellen wir fest, dass es sich bei jeder Instanz um den ursprünglichen Text handelt. Dies stellt uns schon vor die erste Herausforderung, da die meisten Machine Learning Methoden numerische Daten (in Matrixform) benötigen. Unsere erste Aufgabe ist es daher diese Daten in die entsprechende Form zu bringen. Dazu können wir verschiedene *[Feature Extraction](https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)* Methoden verwenden.

In [ ]:
print(f"{categories[y_train[0]]},", y_train[0], "\n", X_train[0])

## 2a) Vektorisierungsmethoden
Vektorisierung bedeutet nichts anderes als einen Text in Vektorform zu bringen. Dazu gibt es verschiedene Möglichkeiten. Die einfachste ist ein sogenannter [sklearn.feature_extraction.text.CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer). Dabei wird aus dem Text-Datenset ein Vokabular erstellt und die einzelnen Terme werden für jeden Text gezählt. 

Der folgende Code bringt das ursprüngliche Set in Matrixform indem es den `CountVectorizer` verwendet. Die Vektorisierung hat ein Vokabular mit 52426 Einträgen erstellt.  

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

extractor = CountVectorizer()
X_train_vec = extractor.fit_transform(X_train)
print(X_train_vec.shape)

Leider erzeugt dieser Vektorizer auch sehr viele schlechte Features welche nur in einem oder zwei Texten vorkommen. Wir können den Parameter `min_df` erhöhen um zu erzwingen, dass jeder Term (*Token*) mindestens in `min_df` Texten vorkommen muss. Man kann einen Integer Wert verwenden oder einen Float zwischen 0.0 und 1.0, welcher den Prozentsatz angibt. Im folgenden Beispiel ist `min_df=5` und man kann sehen, dass das Vokabular schon etwas kleiner ist (11839 Einträge). 

In [ ]:
extractor = CountVectorizer(min_df=5)
X_train_vec = extractor.fit_transform(X_train)
print(X_train_vec.shape)

Wir haben immer noch das Problem, dass unsere Vektorisierung viele unnütze Features erzeugt. Hier ist ein Code, welcher die *Document Frequency* der einzelnen *Token* ausgibt.


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def sort_dict(d):
    return dict(sorted(d.items(), key=lambda x: x[1]))

def document_frequency(X_train, **kwargs):
    """
    Calculates the document frequency of a given set. 
    """
    m = len(X_train)
    extractor = TfidfVectorizer(**kwargs)
    X_train_vec = extractor.fit_transform(X_train)
    # inverse of idf
    df = (1 + m) / np.exp(extractor.idf_ - 1) - 1 
    df = df.astype(np.int64)
    vocabulary = sort_dict(extractor.vocabulary_)  # sort by index
    return sort_dict({k: df[i] for (i, k) in enumerate(vocabulary.keys())})  # sort by df

def print_df(df, n):
    """
    Prints first `n` items in df.
    """
    for token in list(df.items())[:n]:
        print(token)

print_df(document_frequency(X_train, min_df=3), 40)

Der `CountVectorizer` ist eine sehr einfache Methode der Vektorisierung. Wenn wir `min_df` erhöhen, dann wird das Datenset zwar kleiner, jedoch werden auch viele Terme ignoriert, welche potentielle Informationen enthalten. Wir werden eine etwas kompliziertere Methode verwenden: [*Term Frequency $\times$ inverse Document Frequency*](https://scikit-learn.org/stable/modules/feature_extraction.html#tfidf-term-weighting). Die Implementierung findet sich in der Klasse `TfidfVectorizer`. Mit dieser Methode lassen sich oft bessere Modelle erstellen. Der folgende Code soll den Unterschied zwischen *Term Frequency* und *Document Frequency* noch einmal verdeutlichen.

In [ ]:
import numpy as np
import pandas as pd
from itertools import chain
from collections import defaultdict
df = defaultdict(int)

doc1 = "Cat Dog Raven Cat"
doc2 = "Raven Dog Dog Cow"

tf_doc1 = {term: count for (term, count) in zip(*np.unique(np.array(doc1.split(' ')), return_counts=True))}
tf_doc2 = {term: count for (term, count) in zip(*np.unique(np.array(doc2.split(' ')), return_counts=True))}

for term in chain(tf_doc1.keys(), tf_doc2.keys()):
    df[term] += 1  # 

df = dict(df)
dataframe = pd.DataFrame.from_records([tf_doc1, tf_doc2]).fillna(0).astype(int)
dataframe = dataframe.append(df, ignore_index=True)
dataframe.index = ["tf_doc1", "tf_doc2", "df"]
dataframe


Wir können sehen, dass viele Features nicht besonders aussagekräftig sind. Jedoch beinhalten manche wertvolle Informationen. Zum Beispiel kann man annehmen, dass `120mph` ein Hinweis auf die Kategorie `rec.autos` oder `rec.motorcycles` ist. Anstatt diese Token einfach zu entfernen ist es sinnvoll sie vorher umzuwandeln. Zum Beispiel können wir `120mph` mit `__SPEED__` ersetzen. 
Der folgende Code benutzt eine eigene Implementierung des `tokenizer`. Dieser spaltet einen Text in seine Bestandteile auf und wir können jeden Term einzeln bearbeiten.

- Schreibe eine weitere Regel, welche alle Nummern + `kwh, mhz, miles, k, mi, usd` u.s.w. mit `__UNIT__` ersetzt. Z.B.: `100kwh` mit `__UNIT__`.
- Schreibe eine weitere Regel, welche alle Token die mindestens eine Nummer enthalten mit `__ALPHANUMERIC__` ersetzt.
- Wie viele Werte wurden ersetzt? Dafür kannst du wieder die Funktion `document_frequency` von vorhin verwenden.

In [ ]:
import re

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

reg_tokenizer = RegexpTokenizer(r'(?u)\b\w\w+\b')  # (?u) = unicode, \b = word boundary \w\w+ = word with at least 2 characters
lemmatizer = WordNetLemmatizer()
stopwords_en = set(stopwords.words('english'))


def valid(token):
    return (token not in stopwords_en and token != "")

floating_number = re.compile(r'^[+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?$')
def is_decimal(token):
    return bool(floating_number.match(token))
    
int_number = re.compile(r'^[-+]?[0-9]+$')
def is_integer(token):
    return bool(int_number.match(token))

def is_number(token):
    return is_decimal(token) or is_integer(token)

def is_alphanumeric(token):
    ... # ersetzt ... mit dem richtigen Code
    
def is_year(token):
    return len(token) == 4 and token.startswith(("18", "19", "20")) and is_integer(token)

numberunit = re.compile(r'^\d+[a-zA-Z]+$')
def is_speed(token):
    return bool(numberunit.match(token)) and token.endswith("mph")

def is_unit(token):
    ... # ersetzt ... mit dem richtigen Code


def preprocess_token(token):
    token = token.strip("_").lower()
    if token == "": return ""

    if is_year(token): return "__YEAR__"
    if is_number(token): return "__NUMBER__"
    if is_speed(token): return "__SPEED__"
    # ... Füge deine Regeln hier hinzu
    return token

def tokenizer(text):
    text = reg_tokenizer.tokenize(text)
    text = map(preprocess_token, text)
    text = filter(valid, text)
    return map(lemmatizer.lemmatize, text)

df = document_frequency(X_train, tokenizer=tokenizer, min_df=3)
print_df(df, 40)

In [ ]:
print("__NUMBER__:", df["__NUMBER__"])
print("__SPEED__:", df["__SPEED__"])
print("__YEAR__:", df["__YEAR__"])
print()

## 2b) Kernel Methoden und SVMs für Textklassifikation
- Erstelle jetzt eine Pipeline welche diese `tokenizer` verwendet. Die erste Stufe sollte ein `TfidfVectorizer` sein, gefolgt von einer `SVC`.
- Benutze `GridSearchCV` um nach passenden Hyperparametern zu suchen (ignoriere den Parameter `gamma`, der Defaultwert ist gut genug für den Anfang). Vergleiche verschiedene Kerne (z.B.: `"linear", "rbf"` oder  die `cosine_similarity` Funktion). Verwende den `f1_score` als Metrik. Setze dazu `scoring=score` in `GridSearchCV`. Wenn die Trainingszeit zu lange ist, dann verwende einen niedrigeren Wert für `cv`. Du kannst auch den Wert `n_jobs` erhöhen.

In [ ]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score

def score(model, X, y):
    y_pred = model.predict(X)
    return f1_score(y, y_pred, average='macro')

- Berechne die Wahrheitsmatrix deines Modelles anhand des Trainingssets. 
- Normalisiere die Matrix indem du durch die Summe der Zeilen dividierst und die Diagonale 0 setzt.
- Plotte die normalisierte Matrix. Was kannst du sehen?

In [ ]:
from sklearn.base import clone
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [ ]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(y_true, y_pred):
    fig, ax = plt.subplots(1,1, figsize=(10,10)) 
    norm_conf_mx = ... # Berechne die Wahrheitsmatrix und normalisiere!
    ax.matshow(norm_conf_mx, cmap=plt.cm.gray)
    ticks = list(range(11))
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(categories, rotation='vertical')
    ax.set_yticklabels(categories);
    plt.show()

- Welchen F1-Score erzielst du auf dem Testset?

- Welche Kategorien werden `X_test[13], X_test[42], X_test[66]` und `X_test[333]` zugewiesen

In [ ]:
print(categories[y_test[13]], "\n", X_test[13])
print("--------------")
print(categories[y_test[42]], "\n", X_test[42])
print("--------------")
print(categories[y_test[66]], "\n", X_test[66])
print("--------------")
print(categories[y_test[333]], "\n", X_test[333])

finale Version